In [35]:
# fix django notebook

import os, django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()


# CRUD - Create Read Update Delete
# работа с моделями

# использоваться будет ORM django, поскольку таким образом мы не будем привязаны к одной СУБД

In [2]:
# row1, row2 в данном случае будут являються столбцами в базе
'''
class Example(models.Model):
    row1 = models..
    row2 = models...
'''
# также есть возможность открыть интерактивно консоль django чтобы протестировать возможности ORM
# python manage.py shell

'\nclass Example(models.Model):\n    row1 = models..\n    row2 = models...\n'

In [11]:

Paper.objects.filter(pk__gt=17).order_by('title')

# импортируем модель из проекта
# изначально у нас есть созданное приложение paper в нем есть файл models.py
# в данном файле лежит наша модель, которую мы сейчас импортируем

from paper.models import Paper

In [5]:
x = Paper(title="helloworld", slug="worldworld")

In [7]:
# Казалось бы, должен был бы быть запрос к базе, но это не так
# Создание экземпляра класса еще не означает что сразу все добавится в таблицу
# по умолчанию модели в Django - ленивые это создано для оптимизации SQL запросов (поскольку мы могли бы добавить потом еще что-то)
# x.save() - поместит данную запись в таблицу
from django.db import connection # модуль, который позволит нам просмотреть sql запросы
print(connection.queries)

[]


In [15]:
x.save()
print(connection.queries)

[{'sql': 'INSERT INTO "paper_paper" ("title", "slug", "content", "photo", "time_create", "time_update", "is_published", "cat_id") VALUES (\'helloworld\', \'worldworld\', \'\', \'\', \'2023-07-17 21:42:59.300526\', \'2023-07-17 21:42:59.300526\', 1, NULL) RETURNING "paper_paper"."id"', 'time': '0.016'}, {'sql': 'UPDATE "paper_paper" SET "title" = \'helloworld\', "slug" = \'worldworld\', "content" = \'\', "photo" = \'\', "time_create" = \'2023-07-17 21:42:59.300526\', "time_update" = \'2023-07-17 22:05:34.254255\', "is_published" = 1, "cat_id" = NULL WHERE "paper_paper"."id" = 20', 'time': '0.000'}, {'sql': 'UPDATE "paper_paper" SET "title" = \'helloworld\', "slug" = \'worldworld\', "content" = \'\', "photo" = \'\', "time_create" = \'2023-07-17 21:42:59.300526\', "time_update" = \'2023-07-17 22:06:14.826806\', "is_published" = 1, "cat_id" = NULL WHERE "paper_paper"."id" = 20', 'time': '0.016'}]


In [17]:
# Помимо стандартных атрибутов есть еще атрибут pk который совпадает с значением id
print(x.pk is x.id)

True


In [18]:
# еще один способ создания записи в таблицу
y = Paper()
y.title = 'one'
y.content = 'about'
y.save()
print(connection.queries)

[{'sql': 'INSERT INTO "paper_paper" ("title", "slug", "content", "photo", "time_create", "time_update", "is_published", "cat_id") VALUES (\'helloworld\', \'worldworld\', \'\', \'\', \'2023-07-17 21:42:59.300526\', \'2023-07-17 21:42:59.300526\', 1, NULL) RETURNING "paper_paper"."id"', 'time': '0.016'}, {'sql': 'UPDATE "paper_paper" SET "title" = \'helloworld\', "slug" = \'worldworld\', "content" = \'\', "photo" = \'\', "time_create" = \'2023-07-17 21:42:59.300526\', "time_update" = \'2023-07-17 22:05:34.254255\', "is_published" = 1, "cat_id" = NULL WHERE "paper_paper"."id" = 20', 'time': '0.000'}, {'sql': 'UPDATE "paper_paper" SET "title" = \'helloworld\', "slug" = \'worldworld\', "content" = \'\', "photo" = \'\', "time_create" = \'2023-07-17 21:42:59.300526\', "time_update" = \'2023-07-17 22:06:14.826806\', "is_published" = 1, "cat_id" = NULL WHERE "paper_paper"."id" = 20', 'time': '0.016'}, {'sql': 'INSERT INTO "paper_paper" ("title", "slug", "content", "photo", "time_create", "time_

In [19]:
# Каждый класс модели содержит специальный статический объект - objects
# данный класс берется из базовго класса модели Model и представляет ссылку на специальный класс менеджер
Paper.objects

In [21]:
# у данного менеджера есть много полезных методов
# начнем с метода CREATE

print(len(connection.queries))

Paper.objects.create(title='helloer', slug='da', content='worlder') # выполняя данную строчку у нас запись сразу попадает в базу данных

print(len(connection.queries))

5
6


In [22]:
# выбрать все записи из таблицы
Paper.objects.all()

<QuerySet [<Paper: helloer>, <Paper: one>, <Paper: helloworld>, <Paper: onemore>, <Paper: покамир>, <Paper: приветмир>]>

In [23]:
# выбрать все записи где title = "helloer"
Paper.objects.filter(title='helloer')

<QuerySet [<Paper: helloer>]>

In [24]:
# Задача: выбрать все атрибуты у которых id или pk (тоже самое) равны двум

# в django для этого нужно добавлять к базовым атрибутам следующие суффиксы
# attr__gte - >=
# attr__lte - <=

In [26]:
# выбрать все записи, которые не соответствуют критерию id = 1

Paper.objects.exclude(pk=1)

<QuerySet [<Paper: helloer>, <Paper: one>, <Paper: helloworld>, <Paper: onemore>, <Paper: покамир>, <Paper: приветмир>]>

In [29]:
# получить только одну запись из таблицы
Paper.objects.get(pk=17)

<Paper: приветмир>

In [32]:
# Сортировка записей по определенному полю
Paper.objects.filter(pk__gt=17).order_by('title')

<QuerySet [<Paper: helloer>, <Paper: helloworld>, <Paper: one>, <Paper: onemore>, <Paper: покамир>]>

In [34]:
# обратный порядок сортировки записей относительно поля title
Paper.objects.filter(pk__gt=17).order_by('-title')
# нужно добавить минус

<QuerySet [<Paper: покамир>, <Paper: onemore>, <Paper: one>, <Paper: helloworld>, <Paper: helloer>]>

In [37]:
# меняем записи в базе данных
data = Paper.objects.get(pk=18)

<Paper: покамир>

In [38]:
data.title = "приветмир"

In [39]:
data.save()

In [40]:
data

<Paper: приветмир>

In [42]:
# удаляем запись из базы данных
data = Paper.objects.get(pk=18)
data

<Paper: приветмир>

In [43]:
data.delete()

(1, {'paper.Paper': 1})

In [ ]:
# (1, {'paper.Paper': 1}) нулевой элемент кортежа цифра 1 означает что мы удалили 1 запись